In [1]:
import json
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from scipy.stats import linregress


In [2]:
health_data_path= "Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System (version 1).csv"
read_data = pd.read_csv(health_data_path)
#read_data.head()

In [3]:
email = "lilu216@hotmail.com"
api_key = "baygoose48"

epa_url = "https://aqs.epa.gov/data/api/annualData/byState?email={email}&key={api_key}&param=45201&bdate=20110101&edate=20111231&state=37"
epa_url2= "https://aqs.epa.gov/data/api/list/states?email={email}&key={api_key}"

In [4]:
new_df = pd.DataFrame(read_data[["YearEnd","LocationDesc","Question","Data_Value"]])
clean_df = new_df.loc[new_df.YearEnd == 2017]

In [5]:
final_health_data = clean_df.groupby(["Question", "LocationDesc"]).agg({"Data_Value": ["mean"]}).reset_index().rename(columns={"LocationDesc": "state"}).dropna()
final_health_data.count()

Question            486
state               486
Data_Value  mean    486
dtype: int64

In [6]:
for col in final_health_data.columns:
    print(col)

('Question', '')
('state', '')
('Data_Value', 'mean')


In [7]:
email = "lilu216@hotmail.com"
api_key = "baygoose48"
base_url = "https://aqs.epa.gov/data/api/annualData/byState?"
param = "42602"
bdate = "20180101"
edate = "20181231"

# Sample request
url_alabama_co = f"{base_url}email={email}&key={api_key}&param={param}&bdate={bdate}&edate={edate}&state=01"
response = requests.get(url_alabama_co).json()
#print(json.dumps(response, indent=4, sort_keys=True))

In [8]:
#could compare levels of NO2 across states. compare max dates for when the most pollution is. 
state_query_url = f"{base_url}email={email}&key={api_key}&param={param}&bdate={bdate}&edate={edate}&state="

#need to pick the variables we want the measurements for and create a list for each to add to with a for loop.

states = []
mean = []
max_value = []
# max_date =[]
# std_dev = []
# #prm_exceed = []

state_code = ["01", "02", "04", "05", "06", "08", "09", "10", "11", "12", "13", "15", "16", "17", "18", "19", "20", "21", "22", 
              "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41",
              "42", "44", "45", "46", "47", "48", "49", "50", "51", "53", "54", "55", "56", "66", "72", "78"]

for code in state_code:
    response = requests.get(state_query_url + code).json()
    try:
        states.extend([x['state'] for x in response['Data']])
        mean.extend([x['arithmetic_mean'] for x in response['Data']])
        max_value.extend([x['first_max_value'] for x in response['Data']])
#         max_date.extend([x['first_max_datetime'] for x in response['Data']])
#         std_dev.extend([x['standard_deviation'] for x in response['Data']])
#         prm_exceed.extend([x['primary_exceedance_count'] for x in response['Data']])
    except: 
        print("This state has no data", code)

SSLError: HTTPSConnectionPool(host='aqs.epa.gov', port=443): Max retries exceeded with url: /data/api/annualData/byState?email=lilu216@hotmail.com&key=baygoose48&param=42602&bdate=20180101&edate=20181231&state=36 (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')")))

In [ ]:
#create a data frame with info from your lists to combine into one dictionary
no2_dict = {
    "state": states,
    "No2": mean,
    "max No2": max_value,
#     "standard deviation": std_dev,
#     "date of highest value": max_date,
#     "primary exceedence": prm_exceed,
}

#print(no2_dict)

no_2_df = pd.DataFrame(no2_dict)
no_2_df.head()

In [ ]:
clean_air_df = no_2_df.groupby("state").agg({"No2":["mean"],"max No2":["max"]}).reset_index()
clean_air_df.head()

In [ ]:
final_health_data.head()

In [ ]:
#condense_health = final_health_data.replace(to_replace = "Percent of adults aged 18 years and older who have an overweight classification", value ="overweight").replace(to_replace ="Percent of adults aged 18 years and older who have obesity", value = "obese").replace(to_replace ="Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)", value ="moderate physical activity (cardio)").replace(to_replace ="Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week", value ="high phyical activity (cardio/strength)").replace(to_replace ="Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)", value ="Percent adults getting high physical activity (cardio)").replace(to_replace ="Percent of adults who engage in muscle-strengthening activities on 2 or more days a week", value ="moderate physical activity (strength)").replace(to_replace ="Percent of adults who engage in no leisure-time physical activity", value ="no physical activity").replace(to_replace ="Percent of adults who report consuming fruit less than one time daily", value ="less than one fruit per day").replace(to_replace ="Percent of adults who report consuming vegetables less than one time daily", value ="less than one vegetable per day") 


In [ ]:
ques3 = final_health_data[final_health_data["Question"] == "Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)"]
ques4 = final_health_data[final_health_data["Question"] == "Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week"]
ques5 = final_health_data[final_health_data["Question"] == "Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)"]
ques6 = final_health_data[final_health_data["Question"] == "Percent of adults who engage in muscle-strengthening activities on 2 or more days a week"]
ques7 = final_health_data[final_health_data["Question"] == "Percent of adults who engage in no leisure-time physical activity"]

In [ ]:
q3_merge_df = pd.merge(ques3 ,clean_air_df, on="state")
q4_merge_df = pd.merge(ques4 ,clean_air_df, on="state")
q5_merge_df = pd.merge(ques5 ,clean_air_df, on="state")
q6_merge_df = pd.merge(ques6 ,clean_air_df, on="state")
q7_merge_df = pd.merge(ques7 ,clean_air_df, on="state")

In [ ]:
# #def scatter_plot(x_axis, y_axis, show_regress=True, suptitle="", title="", ylabel="", xlabel=""):
# def scatter_plot(x_axis, y_axis):    
#     plt.scatter(x_axis, y_axis)
#     (slope, intercept) = linregress(x_axis, y_axis)
#     #line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))
#     regress_line = x_axis * slope + intercept
#     plt.plot(x_axis,regress_line,"r-")
#     #plt.annotate(line_eq,(x_axis.mean(),y_axis.min()),fontsize=16,color="red")
# #     x_axis.mean()
# #     y_axis.mean()
# #     plt.xlabel(xlabel)
# #     plt.ylabel(ylabel)
# #     plt.suptitle(suptitle)
# #     plt.title(title)
# #     plt.grid()
#     print(f"The r-squared is: {rvalue}")

In [ ]:
No2_mean_3 = q3_merge_df["No2"].values
prct_mean_3 = q3_merge_df[('Data_Value', 'mean')].values

plt.scatter(prct_mean_3, No2_mean_3)
plt.show()

In [ ]:
# Percent adults moderate physical activity (cardio) vs. No2 levels in the air
# outlier - Puerto Rico (lowest percent who get high physical activity)

No2_mean_4 = q4_merge_df["No2"].values
prct_mean_4 = q4_merge_df[('Data_Value', 'mean')].values

plt.scatter(prct_mean_4, No2_mean_4)
plt.show()


In [ ]:
# Percent adults moderate physical activity (cardio) vs. No2 levels in the air
# Drop the outlier

dropped = final_health_data[final_health_data.state != ["Puerto Rico", "Virgin Islands"]
ques4_dropped = dropped[dropped["Question"] == "Percent of adults who achieve at least 150 minutes a week of moderate-intensity aerobic physical activity or 75 minutes a week of vigorous-intensity aerobic physical activity and engage in muscle-strengthening activities on 2 or more days a week"]
ques5_dropped = dropped[dropped["Question"] == "Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical activity or 150 minutes a week of vigorous-intensity aerobic activity (or an equivalent combination)"]
ques6_dropped = dropped[dropped["Question"] == "Percent of adults who engage in muscle-strengthening activities on 2 or more days a week"]
q4_merge_df2 = pd.merge(ques4_dropped ,clean_air_df, on="state")
q5_merge_df2 = pd.merge(ques5_dropped ,clean_air_df, on="state")
q6_merge_df2 = pd.merge(ques6_dropped ,clean_air_df, on="state")

No2_mean_42 = q4_merge_df2["No2"].values.astype(np.float64)
prct_mean_42 = q4_merge_df2[('Data_Value', 'mean')].values.astype(np.float64)

plt.scatter(prct_mean_42, No2_mean_42)
plt.show()

In [ ]:
# Percent adults getting high physical activity vs No2 levels in the air

q5_merge_df = pd.merge(ques5 ,clean_air_df, on="state")
No2_mean_5 = q5_merge_df["No2"].values 
prct_mean_5 = q5_merge_df[('Data_Value', 'mean')].values
plt.scatter(prct_mean_5, No2_mean_5)
plt.show()

In [ ]:
# Percent adults getting high physical activity vs No2 levels in the air - dropped outlier

No2_mean_52 = q5_merge_df2["No2"] 
prct_mean_52 = q5_merge_df2[('Data_Value', 'mean')]

plt.scatter(prct_mean_52, No2_mean_52)
plt.show()

In [ ]:
No2_mean_6 = q6_merge_df["No2"] 
prct_mean_6 = q6_merge_df[('Data_Value', 'mean')]
plt.scatter(prct_mean_6, No2_mean_6)
plt.show()

In [ ]:
# Percent adults getting moderate physical activity (strength) vs No2 levels in the air - dropped outlier

No2_mean_62 = q6_merge_df2["No2"] 
prct_mean_62 = q6_merge_df2[('Data_Value', 'mean')]
plt.scatter(prct_mean_62, No2_mean_62)
plt.show()


In [ ]:
q7_merge_df = pd.merge(ques7, clean_air_df, on="state")
No2_mean_7 = q7_merge_df["No2"].values 
prct_mean_7 = q7_merge_df[('Data_Value', 'mean')].values
plt.scatter(prct_mean_7, No2_mean_7)
plt.show()

In [ ]:
questions = final_health_data["Question"].unique()
#print(questions)
labels = ["Question 3", "Question 4", "Question 5", "Question 6", "Question 7"]
mean = [prct_mean_3.mean(), prct_mean_4.mean(), prct_mean_5.mean(), prct_mean_6.mean(), prct_mean_7.mean()]
mean

x_axis = np.arange(0, len(labels))
tick_locations = []
for x in x_axis:
    tick_locations.append(x)
    
plt.bar(questions[2:7], mean)
plt.xticks(tick_locations, labels)
plt.show()



In [ ]:
# clean_df = no_3_df.rename(columns={('state', ''):"state",('average', 'mean'):"Average No2 parts per billion",('max value', 'max'):"Max No2"})